In [1]:
# geospatial analysis
import geopandas as gpd
import shapely

In [9]:
# essential variables

# let's try to convert everything to EPSG:5070. For more accurate area measurements

# boundary for state of California
us_states = gpd.read_file("Data/Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp")
us_states.to_crs("EPSG:5070", inplace=True)
ca_state = us_states[us_states["STUSPS"] == "CA"]

# HUC8 subbasins
huc8 = gpd.read_file("Data/Boundaries/HUC8_CONUS/HUC8_US.shp")
huc8.to_crs("EPSG:5070", inplace=True)
huc8['CA'] = huc8["STATES"].map(lambda x: "CA" in x)
huc8 = huc8[huc8["CA"]]

# intersect with California
huc8_ca = gpd.clip(huc8, ca_state)

### Land Restrictions

In summary, we can basically ignore land property restrictions.

We haven't investigated hosting capacity yet.

In [ ]:
# let's not worry about utilities for now, just for time. and I don't think it's the most important restriction
# # utilities map in California
# ca_utilities = gpd.read_file("Data/Boundaries/ElectricLoadServingEntities_IOU_POU_-6992589901327565970.geojson")

In [3]:
# federal, state, local lands
# not too many tribal lands in California so we don't worry about that

# overall takes 30 seconds
us_lands = gpd.read_file("Data/Boundaries/USA_Federal_Lands_1879006603790792540.geojson")

# filter invalid geometries
us_lands = us_lands[~us_lands['geometry'].isna()]
us_lands = us_lands[us_lands['geometry'].is_valid]

us_lands.to_crs("EPSG:5070", inplace=True)

# first filter US lands to California
ca_geometry = ca_state.iloc[0].geometry
us_lands_ca = us_lands[us_lands.intersects(ca_geometry)]

In [ ]:
# cut out US lands from California subbasins

# overall, turns out that no HUC8 (besides some island) is fully contained in federal land.

# takes around 30 seconds
all_us_land = us_lands_ca.union_all()

subbasin_us_land = []

for idx, row in huc8_ca.iterrows():
    if shapely.contains(all_us_land, row['geometry']):
        print(idx)
        subbasin_us_land.append(True)
    else:
        subbasin_us_land.append(False)

huc8_ca["Government_Land"] = subbasin_us_land
huc8_ca = huc8_ca[~huc8_ca["Government_Land"]]

355


### Grid Data (Emissions, Water Scarcity, Cost)

### Solar + Wind Data (Emissions, Water Scarcity, Cost)